In [1]:
from google.colab import files
import io
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
import numpy as np 
!pip install pydicom
import pydicom 
import matplotlib.pyplot as plt
!pip install python-docx
import docx 
import pandas as pd 

drive.mount('/content/drive')

     |████████████████████████████████| 2.0 MB 5.6 MB/s 
     |████████████████████████████████| 5.6 MB 4.5 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184507 sha256=3b4fef15fc3ab0187adadcb0b4a3821577cf313c4a8c517317e16139aef615b7
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx
Mounted at /content/drive


In [2]:
# np.save(r'/content/drive/My Drive/images.npy', images)
# np.save(r'/content/drive/My Drive/labels.npy', labels)
# np.save(r'/content/drive/My Drive/patient_IDs.npy', patient_IDs)

x = np.load(r'/content/drive/My Drive/images.npy')
y = np.load(r'/content/drive/My Drive/labels.npy')
patient_IDs = np.load(r'/content/drive/My Drive/patient_IDs.npy')

## include talking about how our dataset doesn't have enough patients to test on all stages
    # only test on the data with enough patients to be learnable from and include a disclaimer 
# print(len(patient_IDs))

In [3]:
print(np.unique(patient_IDs).shape)

(61,)


In [4]:
indices_to_drop = []

for i in range(len(y)):
    # print(y)
    if y[i] == 'N/A' or y[i] == '4':
        indices_to_drop.append(i)
        # print('true')

print('len x', len(x))
x = [x[i] for i in range(len(x)) if i not in indices_to_drop]
y = [y[i] for i in range(len(y)) if i not in indices_to_drop]
patient_IDs = [patient_IDs[i] for i in range(len(patient_IDs)) if i not in indices_to_drop]
# print('len x', len(x))

# print(np.unique(y, return_counts=True))
# print(np.unique(patient_IDs, return_counts=True))

y_new = []

for i in range(len(y)):
    if y[i] == '1A' or y[i] == '1B':
        y_new.append('1')
    elif y[i] == '2A' or y[i] == '2B':
        y_new.append('2')
    elif y[i] == '3A' or y[i] == '3B' or y[i] == '3':
        y_new.append('3')
    elif y[i] == '4':
        y_new.append('4')
    else:
        print(y_new)
    #     y_new.append('N/A')

# print('len y', len(y))
# print("len y_new", len(y_new))


# (array(['1A', '1B', '2A', '2B', '3A', '3B', 'N/A'], dtype='<U3'), array([1230,  419,  301, 1182,  785,  308,  337]))
y = y_new


# print(np.count(y, return_counts=True))

len x 4686


In [5]:
print(len(x), len(y), len(patient_IDs))

4287 4287 4287


In [6]:
# groups = [int(patient) for patient in data['patient']]
groups = []
# data[0:0]

groups = [int(patient) for patient in patient_IDs]

# groups.append
# for index, row in data.iterrows():
    #  groups.append(int(row['patient']))
#     print(row)
#     groups.append(int(row['patient']))
# groupings by patient ID 
# please see: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupShuffleSplit.html

In [7]:
print(groups)

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 1

In [8]:
## Grouping code 


from google.colab import drive
drive.mount('/content/drive')
import sklearn
from sklearn.model_selection import train_test_split, ShuffleSplit

## Create our training, validation, testing datasets (70-15-15 split)
# [[list of all patient 1], [patient 2], [etc]] ## labels 
# [[patient 21], [patient 3]]

# 1 label, 1 patient ID, [all of the images]


# x = images 
# y = labels
patient_id = patient_IDs

# GroupShuffleSplit
gss = ShuffleSplit(n_splits=1, train_size=.7, random_state=60)
gss.get_n_splits()

# train_idx, test_idx = gss.split(x, y, groups)
for train_idx, test_idx in gss.split(x, y):
    train_idx = train_idx
    test_idx = test_idx
    # print("TRAIN:", train_idx, "TEST:", test_idx)

## Temporarily split into train and test before doing a test/validation split 
print(len(x), len(train_idx))


# x, y, patient_id = np.array(x), np.array(y), np.array(patient_id)
print('line 0')
x, y, patient_id = np.array(x), np.array(y), np.array(patient_id)
print("line 1")
x_train = [x[i] for i in train_idx]
y_train = [y[i] for i in train_idx]
patient_id_train = [patient_id[i] for i in train_idx]
print('done')
# x_train, y_train, patient_id_train = [x[i] for i in train_idx], [y[i] for i in train_idx], [patient_id[i] for i in train_idx]
# x_train = x[train_idx]
print("line2")

# x_train, y_train, patient_id_train = np.array(x)[train_idx], np.array(y)[train_idx], np.array(patient_id)[train_idx]
print('line 3')
### gets to this point ^^

x_test_temp, y_test_temp, patient_id_test_temp = [x[i] for i in test_idx], [y[i] for i in test_idx], [patient_id[i] for i in test_idx]
print(len(x_test_temp))
groups_test = [groups[i] for i in test_idx]


gss = ShuffleSplit(n_splits=1, train_size=.5, random_state=27) # was 43
gss.get_n_splits()

for test_idx, valid_idx in gss.split(x_test_temp, y_test_temp):
    test_idx = test_idx
    valid_idx = valid_idx 

x_valid, x_test = [list(x_test_temp)[i] for i in valid_idx], [list(x_test_temp)[i] for i in test_idx]
y_valid, y_test = [list(y_test_temp)[i] for i in valid_idx], [list(y_test_temp)[i] for i in test_idx]
patient_id_valid, patient_id_test = [list(patient_id_test_temp)[i] for i in valid_idx], [list(patient_id_test_temp)[i] for i in test_idx]

print(len(x_train), len(x_valid), len(x_test))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
4287 3000
line 0
line 1
done
line2
line 3
1287
3000 644 643


# Model code

In [9]:
import tensorflow
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# ^^ transfer learning using resnet 50???

In [10]:
EPOCHS = 30 # was 30 
BS = 16
INIT_LR = 1e-4

# Beginning: 7, 32, 0.001
# End (after tuning): , , 

In [11]:
## tf model 
height, width, depth = 512, 512, 1
n_classes = 3
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1

model.add(Conv2D(64, (3, 3), padding="same", input_shape=inputShape, activation='relu'))

model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding="same",activation='relu'))

model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(n_classes))
model.add(Activation("softmax"))

In [12]:
import sklearn 
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
# LabelBinarizer

In [13]:
# print(y_train)
y_train_bin = LabelBinarizer().fit_transform(y_train)
y_test_bin = LabelBinarizer().fit_transform(y_test)
y_valid_bin = LabelBinarizer().fit_transform(y_valid)
y_valid = y_valid_bin
y_train = y_train_bin
y_test = y_test_bin

In [14]:
print(np.array(x_train, dtype=np.uint8).shape)

print(np.unique(y_test))
print(np.unique(y_train))

x_train = np.array(x_train, dtype=np.uint8)
y_train = np.array(y_train)
x_test = np.array(x_test, dtype=np.uint8)
y_test = np.array(y_test)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)


(3000, 512, 512)
[0 1]
[0 1]
(3000, 512, 512) (3000, 3)
(643, 512, 512) (643, 3)


In [15]:
x_test = np.concatenate([np.array(x_test), np.array(x_valid)])
print(x_test.shape)
y_test = np.concatenate([np.array(y_test), np.array(y_valid)])

(1287, 512, 512)


In [16]:
# train the network
print("[INFO] training network...")
model.compile(loss="categorical_crossentropy", 
              optimizer=Adam(learning_rate=INIT_LR), 
              metrics=["accuracy"])

history = model.fit(x_train, y_train, batch_size=BS,
                    validation_data=(np.array(x_test), np.array(y_test)), #was np.array(x_valid)
                    epochs=EPOCHS, verbose=1, shuffle=True)

[INFO] training network...
Epoch 1/30
188/188 [==============================] - 73s 316ms/step - loss: 31.7083 - accuracy: 0.6307 - val_loss: 31.3607 - val_accuracy: 0.6939
Epoch 2/30
188/188 [==============================] - 56s 298ms/step - loss: 0.8684 - accuracy: 0.8833 - val_loss: 39.7467 - val_accuracy: 0.7591
Epoch 3/30
188/188 [==============================] - 56s 298ms/step - loss: 0.1550 - accuracy: 0.9637 - val_loss: 44.6249 - val_accuracy: 0.7653
Epoch 4/30
188/188 [==============================] - 56s 298ms/step - loss: 0.0880 - accuracy: 0.9787 - val_loss: 58.1091 - val_accuracy: 0.7350
Epoch 5/30
188/188 [==============================] - 56s 299ms/step - loss: 0.0614 - accuracy: 0.9830 - val_loss: 57.4077 - val_accuracy: 0.7545
Epoch 6/30
188/188 [==============================] - 56s 299ms/step - loss: 0.0321 - accuracy: 0.9897 - val_loss: 54.6528 - val_accuracy: 0.7591
Epoch 7/30
188/188 [==============================] - 56s 298ms/step - loss: 0.0302 - accuracy: 

In [17]:
test_preds = model.predict(np.array(x_test))
test_preds = np.argmax(test_preds, axis=1)
y_test_calc = np.argmax(y_test, axis=1)
print(test_preds)
from sklearn.metrics import accuracy_score

print("Test accuracy: ", accuracy_score(y_test_calc, test_preds))


[0 0 0 ... 0 2 2]
Test accuracy:  0.797979797979798
